In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import auc,f1_score,recall_score,precision_score,accuracy_score
from keras.models import Sequential 
from keras.layers import Dense , Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

!pip install ipython-autotime
%load_ext autotime

time: 194 µs (started: 2021-04-15 13:56:01 +00:00)


In [3]:
data = pd.read_csv('/content/G4_blood-donation.csv')

labels = ['Months since Last Donation',	'Number of Donations',	'Total Volume Donated (c.c.)',	'Months since First Donation'	,'Made Donation in March 2007']
data = data.rename(columns={'V1':labels[0],'V2':labels[1],'V3':labels[2],'V4':labels[3],'Class':labels[4]},errors="raise")

#Drop o volume
data.drop(columns='Total Volume Donated (c.c.)', inplace= True)

#Transformação de 1 e 2 para 0 e 1
data['Made Donation in March 2007'] = (data['Made Donation in March 2007']>1)*1

data.head()

,Months since Last Donation,Number of Donations,Months since First Donation,Made Donation in March 2007
0,2,50,98,1
1,0,13,28,1
2,1,16,35,1
3,2,20,45,1
4,1,24,77,0


In [4]:
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, stratify = Y,  random_state= 6)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state= 6)

In [5]:
# Normalizando X_train
 
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
 
# Normalizando o resto
 
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [7]:
def rede_neural (neuron_1,neuron_2,neuron_3, activation,kernel_initializer,lr):
    classificador = Sequential()
    classificador.add(Dense(units = neuron_1, activation = activation, input_dim = 3))
    classificador.add(Dense(units = neuron_2, activation = activation))
    classificador.add(Dense(units = neuron_3, activation = activation))
    classificador.add(Dense(units= 1 , activation = 'sigmoid'))
    optmizer = Adam(learning_rate = lr)
    classificador.compile(optimizer = optmizer, loss='binary_crossentropy',
                          metrics = ['accuracy'])
    return classificador

In [8]:
classificador = KerasClassifier(build_fn= rede_neural,verbose=0)
Parametros= {'neuron_1':[2,4,8],
             'neuron_2':[2,4,8],
             'neuron_3':[2,4],
             'activation':['relu'],
             'kernel_initializer' : ['random_uniform'], 
             'epochs':[100,250,500,750],
             'lr':[0.01,0.001]}
 


In [9]:
Parametros


{'activation': ['relu'],
 'epochs': [100, 250, 500, 750],
 'kernel_initializer': ['random_uniform'],
 'lr': [0.01, 0.001],
 'neuron_1': [2, 4, 8],
 'neuron_2': [2, 4, 8],
 'neuron_3': [2, 4]}

In [10]:

grid = GridSearchCV(estimator=classificador, 
                    param_grid=Parametros, 
                    cv=5,scoring='f1_macro')

grid_result = grid.fit(X_train, y_train)
# resultados
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Best: 0.598340 using {'activation': 'relu', 'epochs': 500, 'kernel_initializer': 'random_uniform', 'lr': 0.01, 'neuron_1': 8, 'neuron_2': 8, 'neuron_3': 4}


In [15]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.598340 using {'activation': 'relu', 'epochs': 500, 'kernel_initializer': 'random_uniform', 'lr': 0.01, 'neuron_1': 8, 'neuron_2': 8, 'neuron_3': 4}
time: 2.14 ms (started: 2021-04-15 13:56:14 +00:00)


In [14]:
y_true, y_pred = y_val, grid.predict(X_val)
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.97      0.87        87
           1       0.85      0.42      0.57        40

    accuracy                           0.80       127
   macro avg       0.82      0.70      0.72       127
weighted avg       0.81      0.80      0.77       127

time: 59.3 ms (started: 2021-04-15 13:56:05 +00:00)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
